### CCNY CALENDER SCRAPE

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [2]:
url = "https://www.ccny.cuny.edu/registrar/fall"

response = requests.get(url)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

In [3]:
tr = soup.find_all('tr')
print(tr)

[<tr>
<th class="text-align-left" scope="col" style="width: 305px;">DATES</th>
<th class="text-align-left" scope="col" style="width: 327px;">DAYS</th>
<th scope="col" style="width: 617px;"> </th>
</tr>, <tr>
<td style="width:305px">
<p><strong>August 01</strong></p>
</td>
<td style="width:327px">
<p><strong>Sunday</strong></p>
</td>
<td style="width:617px">
<p><strong>Application for degree for January and February 2022 begins</strong></p>
</td>
</tr>, <tr>
<td style="width:305px">
<p>August 18</p>
</td>
<td style="width:327px">
<p>Wednesday</p>
</td>
<td style="width:617px">
<p>Last day to apply for Study Abroad</p>
</td>
</tr>, <tr>
<td style="width:305px">
<p><strong>August 24</strong></p>
</td>
<td style="width:327px">
<p><strong>Tuesday</strong></p>
</td>
<td style="width:617px">
<p><strong>Last day of Registration;<br/>
			Last day to file ePermit for the Fall 2021;<br/>
			Last day to drop classes for 100% tuition refund;</strong></p>
</td>
</tr>, <tr>
<td style="width:305px">
<

In [4]:
rows = soup.find_all('tr')
data = []

for row in rows:
    cols = row.find_all('td')
    if len(cols) == 3:  # Ensure there are three columns
        date_text = cols[0].get_text(strip=True)
        dow_text = cols[1].get_text(strip=True)
        explanation_text = cols[2].get_text(strip=True)
        
        # Check for date ranges
        if " - " in date_text:
            # Handle date ranges like "December 15 - 21"
            start_part, end_day = date_text.split(" - ")
            start_date = datetime.strptime(f"{start_part} 2021", "%B %d %Y")
            date_str = f"{start_date.year}-{start_date.month:02d}-({start_date.day} - {end_day})"
        else:
            # Check if the date already includes a year (e.g., "January 1, 2022")
            if "," in date_text:
                date = datetime.strptime(date_text, "%B %d, %Y")
            else:
                # Handle single dates by appending the year 2021
                date = datetime.strptime(f"{date_text} 2021", "%B %d %Y")
            date_str = date.strftime("%Y-%m-%d")

        data.append({"date": date_str, "dow": dow_text, "text": explanation_text})

df = pd.DataFrame(data)

In [5]:
df.set_index("date", inplace=True)

In [14]:

display(df)

,dow,text
date,,
2021-08-01,Sunday,Application for degree for January and February 2022 begins
2021-08-18,Wednesday,Last day to apply for Study Abroad
2021-08-24,Tuesday,Last day of Registration;Last day to file ePermit for the Fall 2021;Last day to drop classes for 100% tuition refund;
2021-08-25,Wednesday,Start of Fall Term;Classes begin;Initial Registration Appeals begin;
2021-08-(25 - 31),Wednesday - Tuesday,Change of program period; late fees apply
2021-08-26,Thursday,Last day for Independent Study
2021-08-28,Saturday,First day of Saturday Classes
2021-08-31,Tuesday,Last day to add a class to an existing enrollment;Last day for 75% tuition refund;Financial Aid Certification Enrollment Status date;Last day to apply for Audit option;Last day to drop without the grade of 'WD';Initial Registration Appeals end;
2021-09-01,Wednesday,Verification of Enrollment rosters available to faculty;Course Withdrawal drop period begins (A grade of 'WD' is assigned to students who officially drop a class);
